<a href="https://colab.research.google.com/github/koll-ai/stable-diffusion-qrcode/blob/master/demo_sdqrcode_diffusers_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install git+https://github.com/koll-ai/stable-diffusion-qrcode
!pip install git+https://github.com/holwech/diffusers
!rm -rf stable-diffusion-qrcode
!git clone https://github.com/koll-ai/stable-diffusion-qrcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/koll-ai/stable-diffusion-qrcode to /tmp/pip-req-build-sjb8b_ou
  Running command git clone --filter=blob:none --quiet https://github.com/koll-ai/stable-diffusion-qrcode /tmp/pip-req-build-sjb8b_ou
  Resolved https://github.com/koll-ai/stable-diffusion-qrcode to commit a05fdee9468fb7252570e33622b0b94b07421174
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/holwech/diffusers to /tmp/pip-req-build-9zsbnzdb
  Running command git clone --filter=blob:none --quiet https://github.com/holwech/diffusers /tmp/pip-req-build-9zsbnzdb
  Resolved https://github.com/holwech/diffusers to commit 5257fec4577d550139ff96ce28dd4c16ecd57642
  Installing build dependencies

In [6]:
import sdqrcode

In [ ]:
generator = sdqrcode.init(config = "./stable-diffusion-qrcode/src/sdqrcode/configs/img2img_tile_diffusers.yaml")
# generator = sdqrcode.init(config = "./stable-diffusion-qrcode/src/sdqrcode/configs/default_diffusers.yaml")


unet/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
images = generator.generate_sd_qrcode(height=768, width=768)

  0%|          | 0/16 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/sdqrcode/sdqrcode.py:121 in generate_sd_qrcode           │
│                                                                                                  │
│   118 │   │   │   │   cn_input_img = read_image(unit["cn_input_image"])                          │
│   119 │   │   │   controlnet_input_images.append(cn_input_img)                                   │
│   120 │   │                                                                                      │
│ ❱ 121 │   │   sd_qr_imgs = self.engine.generate_sd_qrcode(input_image, controlnet_input_images   │
│   122 │   │   return sd_qr_imgs                                                                  │
│   123                                                                                            │
│   124                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/sdqrcode/Engines/DiffusersEngine.py:112 in               │
│ generate_sd_qrcode                                                                               │
│                                                                                                  │
│   109 │   │   │   )                                                                              │
│   110 │   │                                                                                      │
│   111 │   │   if self.config["global"]["mode"] == "img2img":                                     │
│ ❱ 112 │   │   │   r = self.pipeline(                                                             │
│   113 │   │   │   │   prompt=self.config["global"]["prompt"],                                    │
│   114 │   │   │   │   image=input_image,                                                         │
│   115 │   │   │   │   negative_prompt=self.config["global"]["negative_prompt"],                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/pipelines/controlnet/pipeline_controlnet_img2i │
│ mg.py:1044 in __call__                                                                           │
│                                                                                                  │
│   1041 │   │   │   │   mid_block_res_sample = None                                               │
│   1042 │   │   │   │                                                                             │
│   1043 │   │   │   │   if isinstance(controlnet, MultiContr

In [ ]:
images[0]

In [ ]:
# !pip install opencv-python transformers accelerate
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import numpy as np
import torch

import cv2
from PIL import Image

# download an image
image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
)
np_image = np.array(image)

# get canny image
np_image = cv2.Canny(np_image, 100, 200)
np_image = np_image[:, :, None]
np_image = np.concatenate([np_image, np_image, np_image], axis=2)
canny_image = Image.fromarray(np_image)

# load control net and stable diffusion v1-5
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny")
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet
)

# speed up diffusion process with faster scheduler and memory optimization
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# generate image
generator = torch.manual_seed(0)
image = pipe(
    "futuristic-looking woman",
    num_inference_steps=20,
    generator=generator,
    image=image,
    control_image=canny_image,
).images[0]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


  0%|          | 0/16 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


In [ ]:
pipe.enable_model_cpu_offload()

image = pipe(
    "futuristic-looking woman",
    num_inference_steps=20,
    generator=generator,
    image=image,
    control_image=canny_image,
).images[0]

  0%|          | 0/16 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/pipelines/controlnet/pipeline_controlnet_img2i │
│ mg.py:1055 in __call__                                                                           │
│                                                                                                  │
│   1052 │   │   │   │   │   │   controlnet_guidance_skip=controlnet_guidance_skip,                │
│   1053 │   │   │   │   │   )                                                                     │
│   1054 │   │   │   │   elif not controlnet_guidance_skip[0]:                                     │
│ ❱ 1055 │   │   │   │   │   down_block_res_samples, mid_block_res_sample = self.controlnet(       │
│   1056 │   │   │   │   │   │   control_model_input,                                              │
│   1057 │   │   │   │   │   │   t,                                                                │
│   1058 │   │   │   │   │   │   encoder_hidden_states=controlnet_prompt_embeds,                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/diffusers/models/controlnet.py:520 in forward            │
│                                                                                                  │
│   517 │   │   # 2. pre-process                                                                   │
│   518 │   │   sample = self.conv_in(sample)                                                      │
│   519 │   │                                                                                      │
│ ❱ 520 │   │   controlnet_cond = self.controlnet_cond_embedd